In [1]:
# !pip install requests-html selenium arsenic pandas

##  Sync vs Async

The Chess Game Analogy

Consecutive vs Concurrent

In [2]:
%%time

import time

iteration_times = [1, 3, 2, 4]


def sleeper(seconds, i=-1):
    if i != -1:
        print(f"{i}\t{seconds}s")
    time.sleep(seconds)


def run():
    for i, second in enumerate(iteration_times):
        sleeper(second, i=i)
    
# run()

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs


In [3]:
start = time.time()
iteration_times = [1, 3, 2, 1]
import asyncio

async def a_sleeper(seconds, i=-1):
    if i != -1:
        print(f"{i}\t{seconds}s")
    await asyncio.sleep(seconds) # coroutine
    
    ellap = time.time() - start
    print(f"{i} done {ellap}")
    return "abc"

async def a_run():
    results = []
    for i, second in enumerate(iteration_times):
        results.append(
            asyncio.create_task(a_sleeper(second, i=i))
        )
    return results
    
results = await a_run()
print(results)
end = time.time() - start

print(end)

[<Task pending name='Task-2' coro=<a_sleeper() running at <ipython-input-3-b040379a2390>:5>>, <Task pending name='Task-3' coro=<a_sleeper() running at <ipython-input-3-b040379a2390>:5>>, <Task pending name='Task-4' coro=<a_sleeper() running at <ipython-input-3-b040379a2390>:5>>, <Task pending name='Task-5' coro=<a_sleeper() running at <ipython-input-3-b040379a2390>:5>>]
0.0006971359252929688
0	1s
1	3s
2	2s
3	1s


## Blocking & Timeouts

In [4]:
def sleeper(seconds, i=-1):
    if i != -1:
        print(f"{i}\t{seconds}s")
    time.sleep(seconds)

sleeper(12)

In [5]:
async def asleeper(seconds, i=-1):
    # time.sleep(seconds)
    if i != -1:
        print(f"a{i}\t{seconds}s")
    await asyncio.sleep(seconds)
    
await asleeper(12)

0 done 12.018728017807007
3 done 12.018839120864868
2 done 12.01893401145935
1 done 12.018996000289917


In [6]:
print("hello word")

hello word


In [7]:
loop = asyncio.get_event_loop()
# loop = asyncio.new_event_loop()
# aysncio.run()


loop.create_task(asleeper(123))

<Task pending name='Task-7' coro=<asleeper() running at <ipython-input-5-f7aa28347698>:1>>

In [8]:
print("hello word")

hello word


In [9]:
done, pending = await asyncio.wait([asleeper(1), asleeper(123)], timeout=2)
done, pending

({<Task finished name='Task-10' coro=<asleeper() done, defined at <ipython-input-5-f7aa28347698>:1> result=None>},
 {<Task pending name='Task-9' coro=<asleeper() running at <ipython-input-5-f7aa28347698>:5> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7fcf20232b20>()]>>})

In [10]:
done

{<Task finished name='Task-10' coro=<asleeper() done, defined at <ipython-input-5-f7aa28347698>:1> result=None>}

In [11]:
pending

{<Task pending name='Task-9' coro=<asleeper() running at <ipython-input-5-f7aa28347698>:5> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7fcf20232b20>()]>>}

In [12]:
# await asyncio.wait(pending)

In [13]:
# await asyncio.wait_for(asleeper(5), timeout=3)

In [14]:
try:
    await asyncio.wait_for(asleeper(5), timeout=3)
except asyncio.TimeoutError:
    print("Task failed")

Task failed


In [15]:
async def asleeper_timeout(seconds, i=-1, timeout=4):
    # time.sleep(seconds)
    if i != -1:
        print(f"a{i}\t{seconds}s")
    await asyncio.wait_for(asyncio.sleep(seconds), timeout=timeout)
    
# await asleeper_timeout(12, timeout=1)

## Scraping with Selenium - Synchronous
New to selenium and web scraping? Watch [this series](https://kirr.co/dwy90n).

In [16]:
url = 'https://www.spoonflower.com/en/shop?on=fabric'

In [17]:
import re
import requests
from requests_html import HTML
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [18]:
def scraper(url):
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    return driver.page_source


# /en/fabric/7137786-genevieve-floral-by-crystal_walen
def extract_id_slug(url_path):
    regex = r"^[^\s]+/(?P<id>\d+)-(?P<slug>[\w_-]+)$"
    group = re.match(regex, url_path)
    if not group:
        return None, None
    return group['id'], group['slug']

In [19]:
content = scraper(url)

In [20]:
html_r = HTML(html=content)

fabric_links = [x for x in list(html_r.links) if x.startswith("/en/fabric")]

datas = []
for path in fabric_links:
    id_, slug_ = extract_id_slug(path)
    print(id_, slug_)
    data = {
        "id": id_,
        "slug": slug_,
        "path": path,
        "scraped": 0 # True / False -> 1 / 0 
    }
    datas.append(data)

5544045 napoleonic-bees-faux-gilt-on-blackest-black-by-peacoquettedesigns
6079351 josie-meadow-floral-by-sweeterthanhoney
5939834 classic-leopard-by-cinneworthington
2623792 purple-space-stars-small-print-by-inspirationz
8618597 rainbow-watercolor-pawprints-by-dragonstarart
6573088 whale-s-song-by-katherine_quinn
5247883 hexo-blue-med-by-nouveau_bohemian
1112778 rosie-riveter-by-spacefem
6327300 call-mountains-evergreen-med-by-nouveau_bohemian
4270747 happy-hair-stylist-friends-blue-by-clayvision_-_ahappybluetree
2330040 maryland-flags-by-elramsay
5700186 puzzle-hearts-by-designedbygeeks
8197261 night-sky-stars-midnight-blue-by-at_the_cottage
6178734 fable-floral-blush-med-by-nouveau_bohemian
6715163 8-wild-heart-florals-white-by-shopcabin
4995555 heart-health-awareness-light-gray-large-by-ohdarkthirty
1306112 opal-by-peacoquettedesigns
5131007 scandinavian-sweet-hedgehog-illustration-kids-gender-neutral-black-white-by-littlesmilemakers
1096407 skull-wall-by-ben_goetting
6650975 love-n

In [21]:
df = pd.DataFrame(datas)
df.head()

,id,slug,path,scraped
0,5544045,napoleonic-bees-faux-gilt-on-blackest-black-by...,/en/fabric/5544045-napoleonic-bees-faux-gilt-o...,0
1,6079351,josie-meadow-floral-by-sweeterthanhoney,/en/fabric/6079351-josie-meadow-floral-by-swee...,0
2,5939834,classic-leopard-by-cinneworthington,/en/fabric/5939834-classic-leopard-by-cinnewor...,0
3,2623792,purple-space-stars-small-print-by-inspirationz,/en/fabric/2623792-purple-space-stars-small-pr...,0
4,8618597,rainbow-watercolor-pawprints-by-dragonstarart,/en/fabric/8618597-rainbow-watercolor-pawprint...,0


In [22]:
df.to_csv("local.csv", index=False)

In [23]:
pd.read_csv("local.csv")

,id,slug,path,scraped
0,5544045,napoleonic-bees-faux-gilt-on-blackest-black-by...,/en/fabric/5544045-napoleonic-bees-faux-gilt-o...,0
1,6079351,josie-meadow-floral-by-sweeterthanhoney,/en/fabric/6079351-josie-meadow-floral-by-swee...,0
2,5939834,classic-leopard-by-cinneworthington,/en/fabric/5939834-classic-leopard-by-cinnewor...,0
3,2623792,purple-space-stars-small-print-by-inspirationz,/en/fabric/2623792-purple-space-stars-small-pr...,0
4,8618597,rainbow-watercolor-pawprints-by-dragonstarart,/en/fabric/8618597-rainbow-watercolor-pawprint...,0
...,...,...,...,...
79,7984649,8-love-care-medical-black-by-rebelmod,/en/fabric/7984649-8-love-care-medical-black-b...,0
80,6812243,cute-kawaii-sushi-small-size-by-penguinhouse,/en/fabric/6812243-cute-kawaii-sushi-small-siz...,0
81,5588706,black-lives-matter-small-scale-by-ashleysummer...,/en/fabric/5588706-black-lives-matter-small-sc...,0
82,6444170,catching-fireflies-by-thestorysmith,/en/fabric/6444170-catching-fireflies-by-thest...,0


## Asynchronous Scraping with `chromedriver` and `arsenic`

[arsenic Docs](https://arsenic.readthedocs.io/en/latest/)

In [24]:
# !pip install arsenic

In [25]:
%%writefile async_scrape.py

import os
import asyncio
from arsenic import get_session, keys, browsers, services
import pandas as pd
from requests_html import HTML
import itertools
import re
import time
import pathlib


# /en/fabric/7137786-genevieve-floral-by-crystal_walen
async def extract_id_slug(url_path):
    regex = r"^[^\s]+/(?P<id>\d+)-(?P<slug>[\w_-]+)$"
    group = re.match(regex, url_path)
    if not group:
        return None, None
    return group['id'], group['slug']



async def get_links(body_content):
    html_r = HTML(html=body_content)
    fabric_links = [x for x in list(html_r.links) if x.startswith("/en/fabric")]
    datas = []
    for path in fabric_links:
        id_, slug_ = await extract_id_slug(path)
        data = {
            "id": id_,
            "slug": slug_,
            "path": path,
            "scraped": 0 # True / False -> 1 / 0 
        }
        datas.append(data)
    return datas

async def scraper(url):
    service = services.Chromedriver()
    browser = browsers.Chrome(chromeOptions={
        'args': ['--headless', '--disable-gpu']
    })
    async with get_session(service, browser) as session:
        await session.get(url)
        body = await session.get_page_source()
        # print(body)
        return body


async def store_links_as_df_pickle(datas=[], name='links.pkl'):
    df = pd.DataFrame(datas)
    df.set_index('id', drop=True, inplace=True)
    df.to_pickle(name)
    return df
    
    
async def run(url):
    body_content = await scraper(url)
    links = await get_links(body_content)
    df = await store_links_as_df_pickle(links)
    return links
    
if __name__ == "__main__":
    url = 'https://www.spoonflower.com/en/shop?on=fabric'
    results = asyncio.run(run(url))
    print(results)


Overwriting async_scrape.py


In [26]:
!python async_scrape.py

Starting ChromeDriver 84.0.4147.30 (48b3e868b4cc0aa7e8149519690b6f6949e110a8-refs/branch-heads/4147@{#310}) on port 60326
Only local connections are allowed.
Please see https://chromedriver.chromium.org/security-considerations for suggestions on keeping ChromeDriver safe.
ChromeDriver was started successfully.
2020-08-04 11:44.13 request                        body={"desiredCapabilities": {"browserName": "chrome", "chromeOptions": {"args": ["--headless", "--disable-gpu"]}}} method=POST url=http://localhost:60326/session
2020-08-04 11:44.14 response                       body={"desiredCapabilities": {"browserName": "chrome", "chromeOptions": {"args": ["--headless", "--disable-gpu"]}}} data={'sessionId': 'b5a19ddf4eea57cd3bf6ff559e811319', 'status': 0, 'value': {'acceptInsecureCerts': False, 'acceptSslCerts': False, 'applicationCacheEnabled': False, 'browserConnectionEnabled': False, 'browserName': 'chrome', 'chrome': {'chromedriverVersion': '84.0.4147.30 (48b3e868b4cc0aa7e8149519690b6f6

2020-08-04 11:44.16 response                       body=None data={'sessionId': 'b5a19ddf4eea57cd3bf6ff559e811319', 'status': 0, 'value': '<html class="js" lang="en" data-wf-page="5d63f10f2f49481077e21b0c" data-wf-site="5c083293d3df665a93b5c3a0"><head><script type="text/javascript" src="https://bam.nr-data.net/1/331b4fd22c?a=15371075&amp;v=1173.01dd3ba&amp;to=IFxcFkBbCFgEE01DBwJQRk1BXAtE&amp;rst=2495&amp;ck=1&amp;ref=https://www.spoonflower.com/en/shop&amp;ap=33&amp;be=217&amp;fe=2443&amp;dc=678&amp;af=err,xhr,stn,ins,spa&amp;perf=%7B%22timing%22:%7B%22of%22:1596566654105,%22n%22:0,%22f%22:0,%22dn%22:41,%22dne%22:42,%22c%22:42,%22s%22:55,%22ce%22:80,%22rq%22:80,%22rp%22:199,%22rpe%22:201,%22dl%22:205,%22di%22:678,%22ds%22:678,%22de%22:680,%22dc%22:2442,%22l%22:2442,%22le%22:2482%7D,%22navigation%22:%7B%7D%7D&amp;fp=1177&amp;fcp=1177&amp;jsonp=NREUM.setToken"></script><script async="" src="https://cdn2.exitintel.com/api/v1/pub/359?callback=exitintel.campaign.campaignsFileLoaded"></scrip

2020-08-04 11:44.16 response                       body=None data={'sessionId': 'b5a19ddf4eea57cd3bf6ff559e811319', 'status': 0, 'value': None} method=DELETE response=<ClientResponse(http://localhost:60326/session/b5a19ddf4eea57cd3bf6ff559e811319) [200 OK]>
<CIMultiDictProxy('Content-Length': '72', 'Content-Type': 'application/json; charset=utf-8')>
 url=http://localhost:60326/session/b5a19ddf4eea57cd3bf6ff559e811319
[{'id': '3840217', 'slug': 'nurse-theme-by-hot4tees_bg-yahoo_com', 'path': '/en/fabric/3840217-nurse-theme-by-hot4tees_bg-yahoo_com', 'scraped': 0}, {'id': '2632362', 'slug': 'colorful-happy-smiley-face-squares-large-print-by-inspirationz', 'path': '/en/fabric/2632362-colorful-happy-smiley-face-squares-large-print-by-inspirationz', 'scraped': 0}, {'id': '7137786', 'slug': 'genevieve-floral-by-crystal_walen', 'path': '/en/fabric/7137786-genevieve-floral-by-crystal_walen', 'scraped': 0}, {'id': '4352750', 'slug': 'loteria-by-jellymania', 'path': '/en/fabric/4352750-loteria-b

In [27]:
name = 'links.pkl'
df = pd.read_pickle(name)
df.head()

,slug,path,scraped
id,,,
3840217,nurse-theme-by-hot4tees_bg-yahoo_com,/en/fabric/3840217-nurse-theme-by-hot4tees_bg-...,0
2632362,colorful-happy-smiley-face-squares-large-print...,/en/fabric/2632362-colorful-happy-smiley-face-...,0
7137786,genevieve-floral-by-crystal_walen,/en/fabric/7137786-genevieve-floral-by-crystal...,0
4352750,loteria-by-jellymania,/en/fabric/4352750-loteria-by-jellymania,0
5033660,80s-accessories-by-diannemehta,/en/fabric/5033660-80s-accessories-by-diannemehta,0


In [28]:
df.shape

(84, 3)

## Hide `arsenic` Logs

In [29]:
!pip install structlog

In [30]:
%%writefile async_scrape.py

import os
import asyncio
from arsenic import get_session, keys, browsers, services
import pandas as pd
from requests_html import HTML
import itertools
import re
import time
import pathlib

import logging
import structlog # pip install structlog

def set_arsenic_log_level(level = logging.WARNING):
    # Create logger
    logger = logging.getLogger('arsenic')

    # We need factory, to return application-wide logger
    def logger_factory():
        return logger

    structlog.configure(logger_factory=logger_factory)
    logger.setLevel(level)


# /en/fabric/7137786-genevieve-floral-by-crystal_walen
async def extract_id_slug(url_path):
    regex = r"^[^\s]+/(?P<id>\d+)-(?P<slug>[\w_-]+)$"
    group = re.match(regex, url_path)
    if not group:
        return None, None
    return group['id'], group['slug']



async def get_links(body_content):
    html_r = HTML(html=body_content)
    fabric_links = [x for x in list(html_r.links) if x.startswith("/en/fabric")]
    datas = []
    for path in fabric_links:
        id_, slug_ = await extract_id_slug(path)
        data = {
            "id": id_,
            "slug": slug_,
            "path": path,
            "scraped": 0 # True / False -> 1 / 0 
        }
        datas.append(data)
    return datas

async def scraper(url):
    service = services.Chromedriver()
    browser = browsers.Chrome(chromeOptions={
        'args': ['--headless', '--disable-gpu']
    })
    async with get_session(service, browser) as session:
        await session.get(url)
        body = await session.get_page_source()
        # print(body)
        return body


async def store_links_as_df_pickle(datas=[], name='links.pkl'):
    df = pd.DataFrame(datas)
    df.set_index('id', drop=True, inplace=True)
    df.to_pickle(name)
    return df
    
    
async def run(url):
    body_content = await scraper(url)
    links = await get_links(body_content)
    df = await store_links_as_df_pickle(links)
    return links
    
if __name__ == "__main__":
    set_arsenic_log_level()
    url = 'https://www.spoonflower.com/en/shop?on=fabric'
    results = asyncio.run(run(url))
    print(results)


Overwriting async_scrape.py


In [31]:
!python async_scrape.py

Starting ChromeDriver 84.0.4147.30 (48b3e868b4cc0aa7e8149519690b6f6949e110a8-refs/branch-heads/4147@{#310}) on port 60388
Only local connections are allowed.
Please see https://chromedriver.chromium.org/security-considerations for suggestions on keeping ChromeDriver safe.
ChromeDriver was started successfully.
Traceback (most recent call last):
  File "async_scrape.py", line 80, in <module>
    results = asyncio.run(run(url))
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/asyncio/runners.py", line 43, in run
    return loop.run_until_complete(main)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/asyncio/base_events.py", line 616, in run_until_complete
    return future.result()
  File "async_scrape.py", line 74, in run
    df = await store_links_as_df_pickle(links)
  File "async_scrape.py", line 66, in store_links_as_df_pickle
    df.set_index('id', drop=True, inplace=True)
  File "/Users/cfe/.local/share/virtualenvs/supercharged-tjIFpWC

## Async Data with Pandas

In [47]:
%%writefile async_scrape.py

import os
import asyncio
from arsenic import get_session, keys, browsers, services
import pandas as pd
from requests_html import HTML
import itertools
import re
import time
import pathlib

import logging
import structlog # pip install structlog

def set_arsenic_log_level(level = logging.WARNING):
    # Create logger
    logger = logging.getLogger('arsenic')

    # We need factory, to return application-wide logger
    def logger_factory():
        return logger

    structlog.configure(logger_factory=logger_factory)
    logger.setLevel(level)


# /en/fabric/7137786-genevieve-floral-by-crystal_walen
async def extract_id_slug(url_path):
    regex = r"^[^\s]+/(?P<id>\d+)-(?P<slug>[\w_-]+)$"
    group = re.match(regex, url_path)
    if not group:
        return None, None
    return group['id'], group['slug']



async def get_links(body_content):
    html_r = HTML(html=body_content)
    fabric_links = [x for x in list(html_r.links) if x.startswith("/en/fabric")]
    datas = []
    for path in fabric_links:
        id_, slug_ = await extract_id_slug(path)
        data = {
            "id": id_,
            "slug": slug_,
            "path": path,
            "scraped": 0 # True / False -> 1 / 0 
        }
        datas.append(data)
    return datas

async def scraper(url):
    service = services.Chromedriver()
    browser = browsers.Chrome(chromeOptions={
        'args': ['--headless', '--disable-gpu']
    })
    async with get_session(service, browser) as session:
        await session.get(url)
        body = await session.get_page_source()
        # print(body)
        return body


def store_links_as_df_pickle(datas=[], name='links.pkl'):
    new_df = pd.DataFrame(datas)
    og_df = pd.DataFrame([{'id': 0}])
    if pathlib.Path(name).exists():
        og_df = pd.read_pickle(name) # read_csv
    df = pd.concat([og_df, new_df])
    df.reset_index(inplace=True, drop=False)
    df = df[['id', 'slug', 'path', 'scraped']]
    df = df.loc[~df.id.duplicated(keep='first')]
    # df.set_index('id', inplace=True, drop=True)
    df.dropna(inplace=True)
    df.to_pickle(name)
    return df
    
    
async def run(url):
    body_content = await scraper(url)
    links = await get_links(body_content)
    return links
    
if __name__ == "__main__":
    set_arsenic_log_level()
    url = 'https://www.spoonflower.com/en/shop?on=fabric'
    name = "link.pkl"
    results = asyncio.run(run(url))
    df = store_links_as_df_pickle(results, name=name)
    print(df.head())


Overwriting async_scrape.py


In [48]:
!python async_scrape.py

Starting ChromeDriver 84.0.4147.30 (48b3e868b4cc0aa7e8149519690b6f6949e110a8-refs/branch-heads/4147@{#310}) on port 60832
Only local connections are allowed.
Please see https://chromedriver.chromium.org/security-considerations for suggestions on keeping ChromeDriver safe.
ChromeDriver was started successfully.
        id  ... scraped
0  4981816  ...     0.0
1  6178734  ...     0.0
2  6852245  ...     0.0
3  4995555  ...     0.0
4  4352750  ...     0.0

[5 rows x 4 columns]


In [49]:
df = pd.read_pickle("link.pkl")
df.shape

(84, 4)

In [50]:
df.head(n=10)

,id,slug,path,scraped
0,4981816,black-white-dogs-by-littleislandcompany,/en/fabric/4981816-black-white-dogs-by-littlei...,0.0
1,6178734,fable-floral-blush-med-by-nouveau_bohemian,/en/fabric/6178734-fable-floral-blush-med-by-n...,0.0
2,6852245,cute-nurse-love-black-no-gradient-by-jannasalak,/en/fabric/6852245-cute-nurse-love-black-no-gr...,0.0
3,4995555,heart-health-awareness-light-gray-large-by-ohd...,/en/fabric/4995555-heart-health-awareness-ligh...,0.0
4,4352750,loteria-by-jellymania,/en/fabric/4352750-loteria-by-jellymania,0.0
5,7137786,genevieve-floral-by-crystal_walen,/en/fabric/7137786-genevieve-floral-by-crystal...,0.0
6,4270747,happy-hair-stylist-friends-blue-by-clayvision_...,/en/fabric/4270747-happy-hair-stylist-friends-...,0.0
7,6079351,josie-meadow-floral-by-sweeterthanhoney,/en/fabric/6079351-josie-meadow-floral-by-swee...,0.0
8,7216659,rainbow-stars-watercolor-abstract-small-by-cry...,/en/fabric/7216659-rainbow-stars-watercolor-ab...,0.0
9,2920223,m81-woodland-camo-by-ricraynor,/en/fabric/2920223-m81-woodland-camo-by-ricraynor,0.0


## Prepare to Scrape Multiple URLs

In [88]:
start = time.time()
iteration_times = [1, 3, 2, 1]


async def asleeper_timeout(seconds, i=-1, timeout=4):
    # time.sleep(seconds)
    if i != -1:
        print(f"a{i}\t{seconds}s")
    await asyncio.wait_for(asyncio.sleep(seconds), timeout=timeout)
    ellap = time.time() - start
    print(f"{i} done {ellap}")


async def a_run():
    results = []
    for i, second in enumerate(iteration_times):
        results.append(
            asyncio.create_task(asleeper_timeout(second, i=i))
        )
    return results
    
results = await a_run()
print(results)
end = time.time() - start

print(end)

[<Task pending name='Task-32' coro=<asleeper_timeout() running at <ipython-input-88-3aebb3aea940>:5>>, <Task pending name='Task-33' coro=<asleeper_timeout() running at <ipython-input-88-3aebb3aea940>:5>>, <Task pending name='Task-34' coro=<asleeper_timeout() running at <ipython-input-88-3aebb3aea940>:5>>, <Task pending name='Task-35' coro=<asleeper_timeout() running at <ipython-input-88-3aebb3aea940>:5>>]
0.0006647109985351562
a0	1s
a1	3s
a2	2s
a3	1s
0 done 1.005234956741333
3 done 1.0053508281707764
2 done 2.0061287879943848
1 done 3.002149820327759


In [103]:
%%writefile async_scrape.py

import os
import asyncio
from arsenic import get_session, keys, browsers, services
import pandas as pd
from requests_html import HTML
import itertools
import re
import time
import pathlib

import logging
import structlog # pip install structlog



def store_links_as_df_pickle(datas=[], name='links.pkl'):
    new_df = pd.DataFrame(datas)
    og_df = pd.DataFrame([{'id': 0}])
    if pathlib.Path(name).exists():
        og_df = pd.read_pickle(name) # read_csv
    df = pd.concat([og_df, new_df])
    df.reset_index(inplace=True, drop=False)
    df = df[['id', 'slug', 'path', 'scraped']]
    df = df.loc[~df.id.duplicated(keep='first')]
    # df.set_index('id', inplace=True, drop=True)
    df.dropna(inplace=True)
    df.to_pickle(name)
    return df


def set_arsenic_log_level(level = logging.WARNING):
    # Create logger
    logger = logging.getLogger('arsenic')

    # We need factory, to return application-wide logger
    def logger_factory():
        return logger

    structlog.configure(logger_factory=logger_factory)
    logger.setLevel(level)


# /en/fabric/7137786-genevieve-floral-by-crystal_walen
async def extract_id_slug(url_path):
    regex = r"^[^\s]+/(?P<id>\d+)-(?P<slug>[\w_-]+)$"
    group = re.match(regex, url_path)
    if not group:
        return None, None
    return group['id'], group['slug']



async def get_links(body_content):
    html_r = HTML(html=body_content)
    fabric_links = [x for x in list(html_r.links) if x.startswith("/en/fabric")]
    datas = []
    for path in fabric_links:
        id_, slug_ = await extract_id_slug(path)
        data = {
            "id": id_,
            "slug": slug_,
            "path": path,
            "scraped": 0 # True / False -> 1 / 0 
        }
        datas.append(data)
    return datas

async def scraper(url, i=-1, timeout=60, start=None):
    service = services.Chromedriver()
    browser = browsers.Chrome(chromeOptions={
        'args': ['--headless', '--disable-gpu']
    })
    async with get_session(service, browser) as session:
        try:
            await asyncio.wait_for(session.get(url), timeout=timeout)
        except asyncio.TimeoutError:
            return []
        body = await session.get_page_source()
        links = await get_links(body)
        if start != None:
            end = time.time() - start
            print(f'{i} took {end} seconds')
        # print(body)
        return links


async def run(urls, timeout=60, start=None):
    results = []
    for i, url in enumerate(urls):
        results.append(
            asyncio.create_task(scraper(url, i=i, timeout=60, start=start))
        )
    list_of_links = await asyncio.gather(*results)
    return list_of_links

if __name__ == "__main__":
    set_arsenic_log_level()
    start = time.time()
    urls = ['https://www.spoonflower.com/en/shop?on=fabric', 
            'https://www.spoonflower.com/en/fabric/6444170-catching-fireflies-by-thestorysmith']
    name = "link.pkl"
    results = asyncio.run(run(urls, start=start))
    print(len(results))
    end = time.time() - start
    print(f'total time is {end}')
#     df = store_links_as_df_pickle(results, name=name)
#     print(df.head())


Overwriting async_scrape.py


In [104]:
!python async_scrape.py

Starting ChromeDriver 84.0.4147.30 (48b3e868b4cc0aa7e8149519690b6f6949e110a8-refs/branch-heads/4147@{#310}) on port 61629
Only local connections are allowed.
Please see https://chromedriver.chromium.org/security-considerations for suggestions on keeping ChromeDriver safe.
ChromeDriver was started successfully.
Starting ChromeDriver 84.0.4147.30 (48b3e868b4cc0aa7e8149519690b6f6949e110a8-refs/branch-heads/4147@{#310}) on port 61630
Only local connections are allowed.
Please see https://chromedriver.chromium.org/security-considerations for suggestions on keeping ChromeDriver safe.
ChromeDriver was started successfully.
0 took 3.004831314086914 seconds
1 took 3.270177125930786 seconds
2
total time is 3.3253772258758545
